In [ ]:
import os
import cv2
import h5py
import random
import numpy as np
from skimage import img_as_float32
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
class SegmentationProcessing(BaseEstimator,TransformerMixin):
    
    def __init__(self, path_in, path_out, attributes):
        self.path_in = path_in
        self.path_out = path_out
        self.attributes = attributes
        self.image_original = None
        self.image_final = None
        self.contador = 0
        self.attr_moment = ""
    
    def fit(seft,*_):
        return self
            
    def run_segmentation(self,*_):
        
        gray = cv2.cvtColor(self.image_original,cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

        # noise removal
        kernel = np.ones((5, 5), np.uint8)
        opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)

        #testando a erosion do background
        img_erosion = cv2.erode(opening, kernel, iterations=1) 

        # Fazer a abertura do background
        img_abertura = cv2.dilate(img_erosion,kernel, iterations=1)

        ###  Suavilição não linear Abertura seguindo de Fechamento
        sua_1 = cv2.dilate(img_abertura, kernel, iterations=1)
        sua_2 = cv2.erode(sua_1,kernel,iterations=1)

        # sure background area
        sure_bg = cv2.dilate(sua_2, kernel, iterations=10)
        
        # removal background in the image
        self.image_final = cv2.bitwise_and(self.image_original,self.image_original,mask = sure_bg)   
        
        # Save the image
        cv2.imwrite("{}/{}/{}.jpg".format(self.path_out,self.attr_moment,self.contador),self.image_final)
        self.contador +=1 
    
    def get_files(self, attribute):
        files = sorted([os.path.join(self.path_in, attribute, file)
                        for file in os.listdir(self.path_in + "/"+attribute)
                        if file.endswith('.jpg')])
        random.shuffle(files)
        return files    
    
    def load_image(self,item):
        
        self.image_original = cv2.imread(item)
        self.image_original = cv2.resize(self.image_original, None, fx=0.5, fy=0.5,interpolation=cv2.INTER_CUBIC)
        self.run_segmentation()
  
    def make_sets(self, *_):
        for attribute in self.attributes:
            self.attr_moment = attribute
            
            dados = self.get_files(attribute)
            
            for item in dados:
                image = self.load_image(item)
            self.contador = 0
            
        
    def transform(self,*_):
        self.make_sets()
        

In [ ]:
path_in = "Imagens/insectos_in"
path_out = "Imagens/insectos_out"
attributes = ["bliPuch","flavi","larg"]
crop = SegmentationProcessing (path_in, path_out, attributes)

In [ ]:
_=crop.transform()